In [ ]:
import numpy as np
import sympy as sm
from sympy.physics.quantum import TensorProduct
from sympy import init_printing
init_printing(use_latex=True, wrap_line=True)
from IPython.display import display
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt
import inspect


# Stokes to correlation mapping

## Direction independent case

Consider the following equation mapping Stokes parameters to visibilities measured between antennas $p$ and $q$

$$ \underset{4\times 1}{V_{pq}} = \underset{4\times 4}{M_{pq}} ~ \underset{4\times 4n}{R_{pq}} ~ \underset{4n\times 4n}{T} ~ \underset{4n\times 1}{B},  $$ 

with $B$ a vector stacking real valued Stokes parameters each of length $n$ and aranged according to

$$B_{4k:4k+4} = \begin{bmatrix} I_k \\ Q_k \\ U_k \\ V_k \end{bmatrix}, \qquad I,Q,U,V \in \mathbb{R}^n.   $$ 

$T$ is a block diagonal matrix which maps Stokes parameters to correlations eg. for linear feeds each block on the diagonal takes the form

$$ T_{4k:4k+4, 4k:4k+4} = \begin{bmatrix} 1 & 1 & 0 & 0 \\ 0 & 0 & 1 & \imath \\ 0 & 0 & 1 & -\imath \\ 1 & -1 & 0   \end{bmatrix}. $$

$R_{pq}$ degrids to the baseline defined by antennas $p$ and $q$. $M_{pq}$ is the Mueller matrix which, in row major form, is defined as

$$ M_{pq} = G_p \otimes G_q^*, \qquad G_p \in \mathbb{C}^{2\times 2},  $$

where $G_p$ denotes the Jones matrix for antenna $p$ and a superscript ${}^*$ denotes complex conjugation.  

Since the noise is Gaussian the data fidelity term for the baseline is given by

$$ \chi^2_{pq} = (V_{pq} - M_{pq} R_{pq} T B)^\dagger \Sigma^{-1}_{pq} (V_{pq} - M_{pq} R_{pq} T B),  $$

where $\Sigma_{pq}$ is the diagonal noise covariance matrix. The gradient of this expression is given by

$$ \nabla_B \chi^2_{pq} = T^\dagger R^\dagger_{pq} M^\dagger_{pq} \Sigma^{-1}_{pq} (V_{pq} - M_{pq} R_{pq} T B), $$

or, equivalently, by

$$ \nabla_B \chi^2_{pq} = T^\dagger R^\dagger_{pq} M^\dagger_{pq} \Sigma^{-1}_{pq} V_{pq} - T^\dagger R^\dagger_{pq} M^\dagger_{pq} \Sigma^{-1}_{pq} M_{pq} R_{pq} T B. $$


If there are no direction dependent effects then, since $T$ is the same for each pixel, we can commute the action of $R_{pq}$ and $T$ to write this as

$$ \chi^2_{pq} = (V_{pq} - M_{pq} T R_{pq} B)^\dagger \Sigma^{-1}_{pq} (V_{pq} - M_{pq} T R_{pq} B),  $$

or equivalently in terms of the Stokes coherencies $C_{pq} = R_{pq} B$ as

$$ \chi^2_{pq} = (V_{pq} - M_{pq} T C_{pq})^\dagger \Sigma^{-1}_{pq} (V_{pq} - M_{pq} T C_{pq}).  $$

In this case, we can solve for the Stokes coherencies by setting the gradient to zero and solving for $C_{pq}$ to find

$$ \hat{C}_{pq} =  W_{pq}^{-1} T^\dagger M^\dagger_{pq} \Sigma^{-1}_{pq} V_{pq},  $$

where the Stokes weights follow from the second derivative and are given by

$$ W_{pq} =  T^\dagger M^\dagger_{pq} \Sigma^{-1}_{pq} M_{pq} T. $$

This gives an equivalent data fidelity term of the form

$$ \chi^2_{pq} = (\hat{C}_{pq} - R_{pq} B)^\dagger W_{pq} (\hat{C}_{pq} - R_{pq} B),  $$

so that gradients can be computed with

$$ \nabla_B \chi^2_{pq} = R^\dagger_{pq} W_{pq} (\hat{C}_{pq} - R_{pq} B).  $$

It is important to note that this can't be done in the presence of direction dependent effects (even trivial ones?) since $T$ does not commute with $R_{pq}$ in that case. 



In [ ]:
# symbolic variables
gp00, gp10, gp01, gp11 = sm.symbols("gp00 gp10 gp01 gp11", real=False)
gq00, gq10, gq01, gq11 = sm.symbols("gq00 gq10 gq01 gq11", real=False)
ep00, ep10, ep01, ep11 = sm.symbols("ep00 ep10 ep01 ep11", real=False)
eq00, eq10, eq01, eq11 = sm.symbols("eq00 eq10 eq01 eq11", real=False)
w0, w1, w2, w3 = sm.symbols("W0 W1 W2 W3", real=True)
v00, v10, v01, v11 = sm.symbols("v00 v10 v01 v11", real=False)
x00, x10, x01, x11 = sm.symbols("x00 x10 x01 x11", real=False)
i, q, u, v = sm.symbols("I Q U V", real=True)

# Jones matrices
Gp = sm.Matrix([[gp00, gp01],[gp10, gp11]])
Gq = sm.Matrix([[gq00, gq01],[gq10, gq11]])

# Mueller matrix (row major form)
# Vpq = Gp Xpq Gq.H
# vec(Vpq) = Gp kron Gq.conj() Xpq = Mpq Xpq
Mpq = TensorProduct(Gp, Gq.conjugate())
Mpqinv = TensorProduct(Gp.inv(), Gq.conjugate().inv())

# inverse noise covariance
Sinv = sm.Matrix([[w0, 0, 0, 0],
                  [0, w1, 0, 0],
                  [0, 0, w2, 0],
                  [0, 0, 0, w3]])
S = Sinv.inv()

# visibilities (row major)
Vpq = sm.Matrix([[v00], [v01], [v10], [v11]])


In [ ]:
# Full Stokes to corr operator for linear pol
T = sm.Matrix([[1, 1, 0, 0],
                [0, 0, 1, 1j],
                [0, 0, 1, -1j],
                [1, -1, 0, 0]])
Tinv = T.inv()

# Full Stokes weights
W = T.H * Mpq.H * Sinv * Mpq * T
Winv = Tinv * Mpqinv * S * Mpqinv.H * Tinv.H

# If the inverse of W computed above is correct we expect to find the idenity for the following
print(sm.simplify(Winv * W))

# Diagonals used as weight
WI = lambdify((gp00, gp01, gp10, gp11, gq00, gq01, gq10, gq11, w0, w1, w2, w3), sm.simplify(W[0,0]))
print(inspect.getsource(WI))
WQ = lambdify((gp00, gp01, gp10, gp11, gq00, gq01, gq10, gq11, w0, w1, w2, w3), sm.simplify(W[1,1]))
print(inspect.getsource(WQ))
WU = lambdify((gp00, gp01, gp10, gp11, gq00, gq01, gq10, gq11, w0, w1, w2, w3), sm.simplify(W[2,2]))
print(inspect.getsource(WU))
WV = lambdify((gp00, gp01, gp10, gp11, gq00, gq01, gq10, gq11, w0, w1, w2, w3), sm.simplify(W[3,3]))
print(inspect.getsource(WV))


In [ ]:
# Full Stokes coherencies
C = Winv * (T.H * (Mpq.H * (Sinv * Vpq)))

I = lambdify((gp00, gp01, gp10, gp11, gq00, gq01, gq10, gq11, w0, w1, w2, w3, v00, v10, v01, v11), sm.simplify(C[0]))
print(inspect.getsource(I))
Q = lambdify((gp00, gp01, gp10, gp11, gq00, gq01, gq10, gq11, w0, w1, w2, w3, v00, v10, v01, v11), sm.simplify(C[1]))
print(inspect.getsource(Q))
U = lambdify((gp00, gp01, gp10, gp11, gq00, gq01, gq10, gq11, w0, w1, w2, w3, v00, v10, v01, v11), sm.simplify(C[2]))
print(inspect.getsource(U))
# TODO - the Stokes V expression seems very different from the others?
V = lambdify((gp00, gp01, gp10, gp11, gq00, gq01, gq10, gq11, w0, w1, w2, w3, v00, v10, v01, v11), sm.simplify(C[3]))
print(inspect.getsource(V))


In [ ]:
# Stokes weights for diagonal gains
Wdiag = W.subs(gp10, 0)
Wdiag = Wdiag.subs(gp01, 0)
Wdiag = Wdiag.subs(gq10, 0)
Wdiag = Wdiag.subs(gq01, 0)

# Diagonals used as weight
WI = lambdify((gp00, gp11, gq00, gq11, w0, w1, w2, w3), sm.simplify(Wdiag[0,0]))
print(inspect.getsource(WI))
WQ = lambdify((gp00, gp11, gq00, gq11, w0, w1, w2, w3), sm.simplify(Wdiag[1,1]))
print(inspect.getsource(WQ))
WU = lambdify((gp00, gp11, gq00, gq11, w0, w1, w2, w3), sm.simplify(Wdiag[2,2]))
print(inspect.getsource(WU))
WV = lambdify((gp00, gp11, gq00, gq11, w0, w1, w2, w3), sm.simplify(Wdiag[3,3]))
print(inspect.getsource(WV))


In [ ]:
# Diag Stokes coherencies
Cdiag = C.subs(gp10, 0)
Cdiag = Cdiag.subs(gp01, 0)
Cdiag = Cdiag.subs(gq10, 0)
Cdiag = Cdiag.subs(gq01, 0)

I = lambdify((gp00, gp11, gq00, gq11, w0, w1, w2, w3, v00, v01, v10, v11), sm.simplify(Cdiag[0]))
print(inspect.getsource(I))
Q = lambdify((gp00, gp11, gq00, gq11, w0, w1, w2, w3, v00, v01, v10, v11), sm.simplify(Cdiag[1]))
print(inspect.getsource(Q))
U = lambdify((gp00, gp11, gq00, gq11, w0, w1, w2, w3, v00, v01, v10, v11), sm.simplify(Cdiag[2]))
print(inspect.getsource(U))
V = lambdify((gp00, gp11, gq00, gq11, w0, w1, w2, w3, v00, v01, v10, v11), sm.simplify(Cdiag[3]))
print(inspect.getsource(V))


It is interesting to note that there are no weights in the above equations. It suggests that the Stokes coherencies should not be computed using a weighted sum as is usually done. The above procedure can be automated by jitting the lambified functions (see below). 

In [ ]:
# an example of how to turn the above expressions into jitted numba functions
from numba import njit
jfn = njit(nogil=True, fastmath=True, inline='always')(I)
t = jfn(1.0+0.1j, 0.1+0.1j, 0.1-0.1j, 1.0-0.1j, 10.0, 5.0, 8.0, 2.0, 1.0+0.1j, 0.1+0.1j, 0.1-0.1j, 1.0-0.1j)
print(t)


# Delay initialisation

Starting with the RIME for an unresolved and unpolarised source at the phase center we have

$$ \mathcal{V}_{pq} = G_p X_{pq} G_q^\dagger,  $$

where the $X_{pq}$ are model visibilities which should contain just the constant amplitude, $A$ say, of the source (why?). Assuming the gains purely consist of delay terms, we can write the visibilities for each correlation as 

$$ \mathcal{V}_{pq} = A \exp(\imath \delta_p \nu) \exp(-\imath \delta_q \nu).  $$

Now we can choose an arbitrary reference antenna, $q=2$ say, and set its phase to zero giving

$$ \mathcal{V}_{p2} = A \exp(\imath \delta_p \nu).  $$

Noting that this is just a complex exponential in frequency, we can take the Fourier transform and read off $\delta_p$ as the coordinate at which the power spectrum peaks

$$ \delta_p = {\tt argmax} PP^*, ~~ {\tt where} ~~ P(\tau) = \int \mathcal{V}_{p2} \exp(-\imath \tau \nu ) d\nu $$

# The Stokes V trick?

The above equations provides the observed Stokes coherencies. Getting rid of the denominator gives (up to a constant scaling)

$$ \mathcal{V}_{pq} = g_{p,00} V_{10} g^*_{q, 11} - g_{p,00} V_{11} g^*_{q, 01} - g_{p,01} V_{10} g^*_{q, 10} + g_{p,01} V_{11} g^*_{q, 00} - 
                 g_{p,10} V_{00} g^*_{q, 11} + g_{p,10} V_{01} g^*_{q, 01} + g_{p,11} V_{00} g^*_{q, 10} - g_{p,11} V_{01} g^*_{q, 00}. $$

Assuming that the cross-hand phase can be parameterized as

$$ X_p = \begin{bmatrix} \exp(\imath \theta) & 0 \\ 0 & 1 \end{bmatrix},  $$

we substitute $g_{p,00} \rightarrow \exp(\imath \theta) g_{p,00}$ (and similar for $q$) to get

$$ \mathcal{V}_{pq} = 
   \exp(\imath \theta) \left(g_{p,00} V_{10} g^*_{q, 11} - g_{p,00} V_{11} g^*_{q, 01}\right) + \exp(-\imath \theta) \left(g_{p,01} V_{11} g^*_{q, 00} - g_{p,11} V_{01} g^*_{q, 00}\right) \\
                 - g_{p,01} V_{10} g^*_{q, 10} - g_{p,10} V_{00} g^*_{q, 11} + g_{p,10} V_{01} g^*_{q, 01} + g_{p,11} V_{00} g^*_{q, 10}.  $$

If $\mathcal{V} = 0$, we can sum over all baselines to find an expression of the form

$$ x \cos(\theta) + \imath y \sin(\theta) = z,  $$

with $x, y$ and $z$ complex numbers given by

$$ x = \sum_{pq} g_{p,00} V_{10} g^*_{q, 11} - g_{p,00} V_{11} g^*_{q, 01} + g_{p,01} V_{11} g^*_{q, 00} - g_{p,11} V_{01} g^*_{q, 00}, $$
$$ y = \sum_{pq} g_{p,00} V_{10} g^*_{q, 11} - g_{p,00} V_{11} g^*_{q, 01} - g_{p,01} V_{11} g^*_{q, 00} + g_{p,11} V_{01} g^*_{q, 00}, $$
$$ z = \sum_{pq} g_{p,01} V_{10} g^*_{q, 10} + g_{p,10} V_{00} g^*_{q, 11} - g_{p,10} V_{01} g^*_{q, 01} - g_{p,11} V_{00} g^*_{q, 10}. $$

Splitting $x = x_r + \imath x_i, y = y_r + \imath y_i$ and $z = z_r + \imath z_i$, gives

$$ (x_r + \imath x_i) \cos(\theta) + \imath (y_r + \imath y_i) \sin(\theta) = z_r + \imath z_i, $$
$$ (x_r\cos(\theta) - y_i\sin(\theta)) + \imath(x_i\cos(\theta) + y_r\sin(\theta)) = z_r + \imath z_i, $$
$$ x_r\cos(\theta) - y_i\sin(\theta) = z_r, \qquad x_i\cos(\theta) + y_r\sin(\theta) = z_i, $$

where we have equated real and imaginary parts. In matrix form

$$ \begin{bmatrix} z_r \\ z_i \end{bmatrix} = \begin{bmatrix} x_r &  -y_i \\ x_i & y_r \end{bmatrix} \begin{bmatrix} \cos(\theta) \\ \sin(\theta)  \end{bmatrix}.  $$

As long as $x_r y_r + x_i y_i \neq 0$ we can solve $\cos(\theta)$ and $\sin(\theta)$ and thus obtain

$$ \theta = \arctan\frac{\sin(\theta)}{\cos(\theta)}.  $$